In [4]:
from typing import Literal, List, Dict, Any, Optional
from typing_extensions import TypedDict
from pydantic import BaseModel, Field, conlist
from langchain_xai import ChatXAI
from langchain_core.tools import Tool
from langchain_core.messages import HumanMessage
from langgraph.graph import MessagesState, StateGraph, START, END
from langgraph.prebuilt import create_react_agent
from langgraph.graph import MessagesState
from langgraph.types import Command

# Define the team members
members = ["content_agent", "image_agent", "notification_agent"]
options = members + ["FINISH"]

# Define the supervisor prompt
with open('prompts/supervisor.txt','r') as f:
    system_prompt = f.read()

# Define the router type for structured output
class Router(TypedDict):
    """Worker to route to next. If no workers needed, route to FINISH."""
    next: Literal[*options]

# Initialize LLM
llm = ChatXAI(model="grok-3-mini-beta")

# Define the state class
class State(MessagesState):
    next: str
    content_json: Dict[str, Any] = None
    images: List[Dict[str, str]] = None
    notification_status: Dict[str, Any] = None
    
# Define the supervisor node function
def supervisor_node(state: State) -> Command[Literal[*members, "__end__"]]:
    messages = [
        {"role": "system", "content": system_prompt},
    ] + state["messages"]
    response = llm.with_structured_output(Router).invoke(messages)
    goto = response["next"]
    if goto == "FINISH":
        goto = END

    return Command(goto=goto, update={"next": goto})

## Build content agent

In [6]:
# Define DuckDuckGo search tool
from langchain_community.tools import DuckDuckGoSearchRun
search_tool = Tool.from_function(
    func=DuckDuckGoSearchRun().run,
    name="duckduckgo_search",
    description="Search the web for information about AI and data science topics. Use this for finding latest information."
)

# Structure output
class SearchDecision(BaseModel):
    performed_search: bool
    search_keywords: List[str] = Field(default_factory=list)
    search_reason: str

class Cover(BaseModel):
    hashtag: str
    heading_line1: str = Field(..., max_length=25)
    heading_line2: str = Field(..., max_length=25)
    grey_box_text: str = Field(..., max_length=30)

class ContentPage(BaseModel):
    title: str = Field(..., max_length=20)
    main_point: str = Field(..., max_length=50)
    illustration_description: str

class EngagementHooks(BaseModel):
    question_for_comments: str
    sharing_incentive: str

class InstagramPost(BaseModel):
    search_decision: SearchDecision
    cover: Cover
    caption: str = Field(..., min_length=300, max_length=500)
    content_pages: conlist(ContentPage, min_length=3, max_length=8)
    engagement_hooks: EngagementHooks
    sources: Optional[List[str]] = None

# Content agent implementation
with open('prompts/content_generation.txt','r') as f:
    content_agent_prompt = f.read()

content_react_agent = create_react_agent(
    llm, 
    tools=[search_tool], 
    prompt=content_agent_prompt,
    response_format=InstagramPost
)

def content_agent_node(state: State) -> Command[Literal["supervisor"]]:
    result = content_react_agent.invoke(state)    
    message_content = result["messages"][-1].content
    content_json = message_content.model_dump()
    return Command(
        update={
            "messages": [
                HumanMessage(content=f"Content generated: Cover with '{content_json.get('cover', {}).get('heading_line1', '')}' and {len(content_json.get('content_pages', []))} content pages.", name="content_agent")
            ],
            "content_json": content_json
        },
        goto="supervisor",
    )


## Image geneartion agent

In [ ]:

# Image agent implementation
with open('prompts/images_generation.txt', 'r') as f:
    image_agent_prompt = f.read()

def image_agent_node(state: State) -> Command[Literal["supervisor"]]:
    # In a real implementation, this would use cairosvg to generate images
    # For this example, we'll simulate the process
    
    content_json = state.get("content_json", {})
    if not content_json or "error" in content_json:
        return Command(
            update={
                "messages": [
                    HumanMessage(content="Error: Invalid content JSON. Cannot generate images.", name="image_agent")
                ],
                "images": []
            },
            goto="supervisor",
        )
    
    # Simulate image generation
    cover_image = {
        "type": "cover",
        "path": "output/cover.png",
        "status": "generated"
    }
    
    content_images = []
    for i, page in enumerate(content_json.get("content_pages", [])):
        content_images.append({
            "type": "content",
            "page_number": i+1,
            "path": f"output/content_{i+1:02d}.png",
            "status": "generated"
        })
    
    all_images = [cover_image] + content_images
    
    return Command(
        update={
            "messages": [
                HumanMessage(content=f"Generated {len(all_images)} images. Cover and {len(content_images)} content pages created successfully.", name="image_agent")
            ],
            "images": all_images
        },
        goto="supervisor",
    )

In [ ]:
# Notification agent implementation
with open('prompts/notification.txt','r') as f:
    notification_agent_prompt = f.read()

def notification_agent_node(state: State) -> Command[Literal["supervisor"]]:
    # In a real implementation, this would send an email
    # For this example, we'll simulate the process
    
    content_json = state.get("content_json", {})
    images = state.get("images", [])
    
    notification_status = {
        "sent": True,
        "recipient": "joseph.hohoyin@gmail.com",
        "subject": "Instagram Content Generation Complete",
        "content_summary": f"Generated Instagram post about {content_json.get('cover', {}).get('heading_line1', 'AI')}",
        "images_attached": len(images)
    }
    
    return Command(
        update={
            "messages": [
                HumanMessage(content=f"Notification sent to {notification_status['recipient']} with {notification_status['images_attached']} attached images.", name="notification_agent")
            ],
            "notification_status": notification_status
        },
        goto="supervisor",
    )

# Build the graph
builder = StateGraph(State)
builder.add_edge(START, "supervisor")
builder.add_node("supervisor", supervisor_node)
builder.add_node("content_agent", content_agent_node)
builder.add_node("image_agent", image_agent_node)
builder.add_node("notification_agent", notification_agent_node)
graph = builder.compile()

In [2]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [3]:

# Example usage
if __name__ == "__main__":
    inputs = {
        "messages": [
            {
                "role": "user", 
                "content": "請為Instagram生成一套關於如何使用AI進行數據分析的輪播貼文"
            }
        ]
    }
    
    for output in graph.stream(inputs):
        if "__end__" not in output:
            step = list(output.keys())[-1]
            print(f"Step: {step}")
            if step in members:
                print(f"Message from {step}: {output[step]['messages'][-1].content}")
                if step == "content_agent" and "content_json" in output[step]:
                    print(f"Generated JSON content with {len(output[step]['content_json'].get('content_pages', []))} pages")
                elif step == "image_agent" and "images" in output[step]:
                    print(f"Generated {len(output[step]['images'])} images")
                elif step == "notification_agent" and "notification_status" in output[step]:
                    print(f"Notification status: {output[step]['notification_status']['sent']}")
    
    print("Process completed.")

Step: supervisor
Step: content_agent
Message from content_agent: Content generated: Cover with 'AI如何改變數據分析' and 6 content pages.
Generated JSON content with 6 pages
Step: supervisor
Step: image_agent
Message from image_agent: Generated 7 images. Cover and 6 content pages created successfully.
Generated 7 images
Step: supervisor
Step: notification_agent
Message from notification_agent: Notification sent to user@example.com with 7 attached images.
Notification status: True
Step: supervisor
Process completed.
